# Measuring COSEBIs from Shear Catalogs

Author: Laila Linke (laila.linke@uibk.ac.at)

This notebook demonstrates how to measure COSEBIs (Complete Orthogonal Sets of E/B-Integrals) from weak lensing shear catalogs. The workflow includes:

- Reading in galaxy shear data from a FITS catalog (as is available from cosmohub).
- Selecting galaxies based on user specified cuts.
- Measuring shear correlation functions (ξ₊, ξ₋) for all tomographic bin combinations using TreeCorr.
- Saving the correlation functions for each bin combination.
- Running an external COSEBIs pipeline to compute COSEBIs modes from the measured correlation functions.

Adjust file paths, column names, and parameters as needed for your dataset.

This code relies on `treecorr` by Mike Jarvis and the COSEBI part of `kcap` by the KiDS team, in particular Marika Asgari.

Before running it, you need to install `treecorr`, using

`pip install treecorr`

and download/clone the folder `cosebis` from `https://github.com/KiDS-WL/kcap/tree/master/cosebis`

In [ ]:
# Necessary imports

from astropy.table import Table
import treecorr
import numpy as np

## 0. User specifications

First you need to set several constants. These are
- The name of the data catalog fits file
- The output directory for shear correlation functions and COSEBIs
- The location of the cosebi scripts from kcap
- The location of the cosebi normalization and root lookup tables from kcap
- The location of the kernel functions for the cosebi calculation from kcap. This folder can be empty, then the kernel functions are calculated on the fly from the normalization and roots
- The number of tomo bins
- The minimal and maximal angular scale for the correlation functions
- The number of bins for the correlation functions (at least 1000 is recommended for COSEBIs)
- The maximum COSEBI mode to compute

In [ ]:
datacatalog = "../data/22028.fits"
out_directory = "../measurements_2pt/"

bin_cosebis_directory = "../cosebis/measurement_pipeline/"
norm_roots_directory = "../cosebis/TLogsRootsAndNorms/"
Ts_directory="../cosebis/Tplus_minus/"

Ntomo = 6 # number of tomographic bins
theta_min = 0.5 # minimum angular separation in arcmin
theta_max = 300 # maximum angular separation in arcmin
Nbins = 1000 # number of angular bins for correlation functions (>=1000 is recommended)
n_max = 20 # maximum COSEBIs mode to compute


## 1. Read in data and make selections
Next, we read in the data and make selections. Here you can specify any additional selections you need and also change the column names if your data uses different ones

In [ ]:
data = Table.read(datacatalog)

In [ ]:
mask = (data['she_lensmc_weight']>0)
# Add more selections here if needed
data = data[mask]

In [ ]:
# Change these column names if your data uses different ones
ra=data['right_ascension']
dec=data['declination']
ra_units='deg'
dec_units='deg'
g1=data['she_lensmc_e1_corrected']
g2=data['she_lensmc_e2_corrected']
w=data['she_lensmc_weight']

## 2. Measure shear correlation functions for each tomo bin combination

Now we calculate the shear correlation functions using treecorr. The results are written to an ascii file for each tomographic bin combination

In [ ]:
for i in range(Ntomo):
    print("Processing tomographic bin combination: ", i, i)
    # Auto-correlations

    # Mask for picking tomographic bin
    mask_tomo=(data['tomographic_bin']==i+1)

    # Create TreeCorr catalog
    cat1 = treecorr.Catalog(ra=ra[mask_tomo], dec=dec[mask_tomo], ra_units=ra_units, 
                            dec_units=dec_units, g1=g1[mask_tomo], g2=g2[mask_tomo], w=w[mask_tomo])

    # Set up the correlation function object
    gg = treecorr.GGCorrelation(min_sep=theta_min, max_sep=theta_max, nbins=Nbins, sep_units='arcmin')

    # Measure the correlation functions
    gg.process(cat1)

    # Save the correlation functions
    np.savetxt(out_directory+"gg_thMin"+str(theta_min)+"_thMax"+str(theta_max)+"_tomo"+str(i)+"_"+str(i)+".dat", 
               np.column_stack([gg.meanr, gg.xip, gg.xim]))
    

    for j in range(i+1, Ntomo):
        print("Processing tomographic bin combination: ", i, j)
        # Cross-correlations

        # Mask for picking tomographic bin
        mask_tomo2=(data['tomographic_bin']==j+1)

        # Create TreeCorr catalog
        cat2 = treecorr.Catalog(ra=ra[mask_tomo2], dec=dec[mask_tomo2], ra_units=ra_units, 
                                dec_units=dec_units, g1=g1[mask_tomo2], g2=g2[mask_tomo2], w=w[mask_tomo2])

        # Set up the correlation function object
        gg = treecorr.GGCorrelation(min_sep=theta_min, max_sep=theta_max, nbins=Nbins, sep_units='arcmin')

        # Measure the correlation functions
        gg.process(cat1, cat2)

        # Save the correlation functions
        np.savetxt(out_directory+"gg_thMin"+str(theta_min)+"_thMax"+str(theta_max)+"_tomo"+str(i)+"_"+str(j)+".dat", 
                   np.column_stack([gg.meanr, gg.xip, gg.xim]))

    


## 3. Measure COSEBIs
Finally, we can convert the correlation functions to COSEBIs. For this we use the kcap routine which we call directly from this notebook for each tomographic bin combination. The resulting COSEBIs are written to file (one for each tomographic bin combination).

In [ ]:
# Format theta_min and theta_max for filenames
theta_min_str=f"{theta_min:.2f}"
theta_max_str=f"{theta_max:.2f}"

In [ ]:
# Now compute COSEBIs for each tomographic bin combination
for i in range(Ntomo):
    for j in range(i, Ntomo):
        print("Processing COSEBIs for tomographic bin combination: ", i, j)

        file=out_directory+"/gg_thMin"+str(theta_min)+"_thMax"+str(theta_max)+"_tomo"+str(i)+"_"+str(j)+".dat"
        outfile="cosebis_thMin"+str(theta_min_str)+"_thMax"+str(theta_max_str)+"_tomo"+str(i)+"_"+str(j)

        !python {bin_cosebis_directory}run_measure_cosebis_cats2stats.py -i {file} --cfoldername {out_directory} -o {outfile} --norm {norms_roots_directory}/Normalization_{theta_min_str}-{theta_max_str}.table -r {norms_roots_directory}/Root_{theta_min_str}-{theta_max_str}.table -b log -s {theta_min_str} -l {theta_max_str} -n {n_max} --tfoldername {Ts_directory}